In [2]:
import numpy as np
from scipy.stats import truncnorm
from six import print_


def multiplyGauss(m1, s1, m2, s2):
    s = 1 / (1 / s1 + 1 / s2)
    m = (m1 / s1 + m2 / s2) * s
    return m, s

def divideGauss(m1, s1, m2, s2):
    m,s =  multiplyGauss(m1, s1, m2, -s2)
    return m, s

def truncGaussMM(a,b,m0,s0):
    a_scaled, b_scaled = (a - m0) / np.sqrt(s0), (b - m0) / np.sqrt(s0)
    m = truncnorm.mean(a_scaled,b_scaled, loc=m0, scale=np.sqrt(s0))
    s = truncnorm.var(a_scaled, b_scaled, loc=m0, scale=np.sqrt(s0))
    return m,s 

In [3]:


m1, s1 =  25, (25/3)**2
m2, s2 = 25, (25/3)**2
sv = (25/6)**2 
y0 = 1
mu_diff_m = m1 - m2
mu_diff_s = s1 + s2 + sv

mu3_m = m1
mu3_s = s1

mu4_m = mu3_m
mu4_s = mu3_s

mu5_m = mu_diff_m 
mu5_s = mu_diff_s 

if y0 == 1:
    a,b = 0, np.inf
else:
    a,b = -np.inf, 0


pt_m, pt_s = truncGaussMM(a, b, mu5_m, mu5_s)
# Compute the message from t to f_3
mu6_m, mu6_s = divideGauss(pt_m, pt_s, mu5_m, mu5_s)
# Compute the message from f_3 to s1
mu7_m = mu6_m  
mu7_s = mu6_s +s1+sv
# Compute the message from f_3 to s2
mu8_m=mu6_m
mu8_s=mu6_s+s2+sv

# Compute the marginal of s1
px_m, px_s = multiplyGauss(mu3_m, mu3_s, mu7_m, mu7_s)
print(f"Mean of s1: {px_m}")
print(f"Variance of s1: {px_s}")


#compute the message from f_3 to s2
mu_8m = mu6_m 
mu_8s = mu6_s 
#marginal of s2
px_m, px_s = multiplyGauss(mu3_m, mu3_s, mu_8m, mu_8s)
print(f"Mean of s2: {px_m}")
print(f"Variance of 2: {px_s}")

Mean of s1: 22.3591389781539
Variance of s1: 49.795686038037616
Mean of s2: 20.91401871338585
Variance of 2: 39.04357954105098
